# <Feedback control of a cassie bipedal robot: walking, standing and riding a Segway> 

## 状态说明
$q_1$: hip roll, $q_2$: hip yaw, $q_3$: hip pitch, $q_4$: knee pitch, $q_5$: shin pitch, $q_6$: tarsus pitch, $q_7$: toe pitch

全维状态量 $q = [q_x, q_y, q_z, q_{yaw}, q_{pitch}, q_{roll}, q_{1L}, q_{2L},q_{3L}, q_{4L}, q_{5L}, q_{6L}, q_{7L}, q_{1R}, q_{2R},q_{3R}, q_{4R}, q_{5R}, q_{6R}, q_{7R}]$

$q_{yaw}, q_{pitch}, q_{roll}$ 根据cassie的IMU进行EKF滤波得到， $q_{1L}, ... q_{7L},q_{1R, ... q_{7R}}$根据电机编码器得到

$q_5, q_6$是被动关节（相当硬的弹簧连接四连杆），弹簧未压缩时：$q_5 = 0, q_6 = -q_4+13^\circ$


## 动力学方程
Lagrange 动力学方程为 

$$ D(q) \ddot{q} + H(q, \dot{q}) = Bu + J_{sp}^T(q) \tau_{sp} + J(q)^T \lambda $$


D(q)为广义质量矩阵， B为控制力矩矩阵，u是控制量，$H(q,\dot(q))$为速度和重力相关项， $J_{sp}(q) = \frac{\partial}{\partial q} \begin{bmatrix} q_{5L} \\ q_{4L}+q_{5L}+q_{6L} \\ q_{5R} \\ q_{4R}+q_{5R}+q_{6R} \end{bmatrix}$，$J_{sp}, \tau_{sp}$分别是弹簧Jacobian和弹簧力矩。$\lambda$是摩擦力（contact wrench）。

## Hybrid model for walking

walking包括单足支撑和双足支撑，双足支撑的时间很短

### 右脚单脚站立模型
这里将足端触地视为完整约束，为$ c_R{q}^T = [p_R^x, p_R^y, p_R^z, \psi_R^{yaw}, \psi_R^{pitch}] $，前三维是右脚在地面位置发生相对滑动，后两项是右脚在地面时pitch和yaw角不发生变化（cassie脚很窄，所以不限制roll角）,此时右脚接触Jacobian为 

$$ J_R(q) = \frac{\partial c_R(q)}{ \partial q} $$

接触动力学为：$J_R(q) \ddot{q} + \dot{J_R}(q, \dot{q}) \dot{q} = 0$，等式右边为0意味着将$c_R(q)$中的状态加速度限制为0。由于接触动力学方程满秩（rank = 5），因此可以将原动力学方程（20维）降秩为15DoF
### 双脚站立模型
摆动相对地高度为0，one-foot-stance变为two-foot-stance。双脚站立模型中左右脚均有完整的接触约束。

Note1：摆动相的脚需要大致与地面平行（$q_7$施加力矩），但是该力矩足够小，所以当机器人落地时，脚会被机器人自身重力克服$q_7$力矩压成贴合地面的角度。

Note2：摆动相的足端是否与地面发生碰撞检测依据：$q_5， q_6$的弹簧是否被压缩。碰撞仿真是通过离散映射的方式实现的，碰撞前后位置连续但是速度不连续。碰撞后，支撑相和摆动相的约束互换。

### 单脚到双脚的动力学切换模型
以$q^-, q^+$分别代表摆动相触地前后的状态，则触地前后的动力学关系满足如下约束，
$$ \begin{bmatrix}  D(q) & -J_L^T(q)\\ J_L(q) & 0 \end{bmatrix} \begin{bmatrix} \dot{q^+} \\ \delta F_L \end{bmatrix} = \begin{bmatrix} D(q)\dot{q^-} \\ 0 \end{bmatrix} $$
其中$\delta F_L$是碰撞冲击向量。因为D(q)满秩并且$J_L(q)$满秩，所以上式左边是可逆的。所以能够根据$q_-$求解$q_+$。
